# Carregamento de pacotes

In [ ]:
import numpy as np
import pandas as pd
import warnings

# Configuração do notebook

In [ ]:
warnings.filterwarnings('ignore')

# Funções

In [ ]:
# Remove o modelo da descrição se estiver repetido
def strip_string_with_prefix_from_text(text, prefix, string):
    text = text.strip()
    prefix = prefix.strip()
    string = string.replace(' ', '')

    length_prefix = len(prefix)
    length_string = len(string)

    if text[0:length_prefix] == prefix:
        text = text[length_prefix:].strip()
    
    string_in_text = text[0:length_string]

    if string_in_text == string:
        return text[length_string + 1:].strip()
    else:
        return text

# Identifica se há indicação de ser hatch ou sedan na descrição e trata o nome do modelo de acordo
def is_hatch_or_sedan(description, model):
    hatch = 'HATCH'
    sedan = 'SEDAN'
    fastback = 'FASTBACK' # Trata exceção do focus
    if description.upper().find(hatch) != -1:
        return model + ' ' + hatch
    elif description.upper().find(sedan) != -1:
        return model + ' ' + sedan
    elif description.upper().find(fastback) != -1 and model == 'FOCUS':
        return model + ' ' + sedan
    else:
        return model
    
# Identifica o tipo de transmissão
def label_transmisson(description, names, label):
    for name in names:
        if description.find(name) != -1:
            return label

def merge_transmission_cols(label1, label2, label3):
    if label1 != None:
        return label1
    elif label2 != None:
        return label2
    else:
        return label3

def write_to_file(file_name, content):
    f = open(file_name, 'w+')
    for line in content:
        f.write(f'{line}\n')
    f.close()

# Carregamento da base de dados

In [ ]:
df_full = pd.read_csv('Extração Zoss 30.07.21.csv')
df_full.info()

In [ ]:
# Amostragem
df = df_full
# df = df_full.head(1000)
# df = df_full.sample(1000)

# Eliminação de qualquer linha com algum valor menor do que zero

In [ ]:
df = df[df.select_dtypes(include=[np.number]).ge(0).all(1)]

# Tratamento da coluna Modelo
* Foi identificado que há duplicidade com o HB20S, tendo ocorrências como 'HB20S' e 'HB20 S'
* O Onix Joy está identificado como Joy apenas

In [ ]:
# Correção da identificação do HB20S
df.replace('HB20 S', 'HB20S', inplace=True)

In [ ]:
# Corrige a identificação do Onix Joy
df['DescricaoSite'] = df['DescricaoSite'].apply(lambda row: 'JOY ' + row if row == 'JOY' else row)
df['Modelo'].replace('JOY', 'ONIX', inplace=True)

# Tratamento da coluna de descrição
* Algumas entradas possuiam o nome do modelo do carro repetido
* Alguns nome de modelo tinham o prefixo 'NEW' e este foi desconsiderado

In [ ]:
# Retira o modelo da descrição
df['Descricao'] = df.apply(lambda row: strip_string_with_prefix_from_text(row['DescricaoSite'], 'NEW ', row['Modelo']), axis=1)

print(df.shape)
df.head()

# Identificação das características dos modelos
* Alguns modelos possuem diferenciação entre hatch e sedan
* Volume do motor
* Tipo de transmissão
* Diesel ou não

In [ ]:
# Identifica se é hatch ou sedan pela descrição
df['Modelo'] = df.apply(lambda row: is_hatch_or_sedan(row['DescricaoSite'], row['Modelo']), axis=1)

In [ ]:
# Identifica como hatch os que tem versão sedan e não são sedan
models = list(pd.Series(df['Modelo'].unique()).values)
for model in models:
    if model + ' SEDAN' in models:
        df['Modelo'].replace(model, model + ' HATCH', inplace=True)

In [ ]:
# Retira a identificação de hatch dos que não tem versão sedan
models = list(pd.Series(df['Modelo'].unique()).values)
for model in models:
    model_unhatched = model.replace(' HATCH', '')
    if (' HATCH' in model) & (model_unhatched + ' SEDAN' not in models):
        df['Modelo'].replace(model, model_unhatched, inplace=True)

In [ ]:
# Identificação do volume do motor
df['VolumeMotor'] = df.apply(lambda row: row['Descricao'][row['Descricao'].find('.') - 1 : row['Descricao'].find('.') + 2], axis=1)
df['VolumeMotor'].replace(r'', 'Eletrico', inplace=True)

In [ ]:
# Identifica o tipo de transmissão
good_automatics = ['AUTOMATICO','DIRECT SHIFT', '4MOTION DSG', 'GEARTRONIC', 'MULTIDRIVE', '9G-TRONIC', 'X-TRONIC', 
                   'S TRONIC','S-TRONIC', '7G-DCT', '4MATIC', 'AT6', 'AT9', 'CVT', 'DCT', 'DSG']
                   
bad_automatics = ['AUTOMATIZADO', 'POWERSHIFT', 'TIPTRONIC', 'I-MOTION', 'EASY-R']

df['TransmissaoATBoa'] = df.apply(lambda row: label_transmisson(row['Descricao'], good_automatics, 'AUTOMATICO BOM'), axis=1)
df['TransmissaoATRuim'] = df.apply(lambda row: label_transmisson(row['Descricao'], bad_automatics, 'AUTOMATICO RUIM'), axis=1)
df['Trasmissao'] = df.apply(lambda row: merge_transmission_cols(row['TransmissaoATBoa'], row['TransmissaoATRuim'], 'MANUAL'), axis=1)
df.drop(['TransmissaoATBoa'], axis=1, inplace=True)
df.drop(['TransmissaoATRuim'], axis=1, inplace=True)

In [ ]:
# Identificação se o motor é Diesel ou não
df['Combustivel'] = df['Descricao'].apply(lambda row: 'DIESEL' if 'DIESEL' in row.upper()
                                                else ('ELETRICO' if 'ELETRICO' in row.upper()
                                                else  'OUTRO'))

# Adequação dos dados ao uso

In [ ]:
# Eliminação das colunas que não serão usadas
df.drop(['CodigoFipe'], axis=1, inplace=True)
df.drop(['RegraKM'], axis=1, inplace=True)
df.drop(['DescricaoSite'], axis=1, inplace=True)
df.drop(['IdAnuncio'], axis=1, inplace=True)
df.drop(['DATA'], axis=1, inplace=True) #Todas as datas são iguais
df.drop(['LinkAnuncio'], axis=1, inplace=True)

# Visualização e armazenamento da base tratada

In [ ]:
print(df.shape)
df.head()

In [ ]:
# Salva arquivo com a base tratada
df.to_csv('base_tratada.csv')